In [1]:
import pandas as pd
from pandas import Series, DataFrame
import json
from janome.tokenizer import Tokenizer
from requests_oauthlib import OAuth1Session
from wordcloud import WordCloud
import emoji
import re

In [29]:
def create_oauth_session(oauth_key_dict):
    oauth = OAuth1Session(
        oauth_key_dict['consumer_key'],
        oauth_key_dict['consumer_secret'],
        oauth_key_dict['access_token'],
        oauth_key_dict['access_token_secret']
    )
    return oauth

def tweet(oauth, text):
    url = 'https://api.twitter.com/1.1/statuses/update.json'
    params = {'status': text + '\n#TweetFromPy'}
    req = oauth.post(url, params)

    if req.status_code == 200:
        print('tweet succeed!')
    else:
        print('tweet failed')
        
def tweet_image(oauth, text, image_file):
    url_media = "https://upload.twitter.com/1.1/media/upload.json"
    url_text = "https://api.twitter.com/1.1/statuses/update.json"
    
    files = {"media" : open(image_file, 'rb')}
    req_media = oauth.post(url_media, files = files)
    if req_media.status_code != 200:
        print ('media upload failed: %s', req_media.text)
        exit()
        
    media_id = json.loads(req_media.text)['media_id']
    print ("Media ID: %d" % media_id)
    
    params = {'status': text + '\n#TweetFromPy', 'media_ids': [media_id]}
    req = oauth.post(url_text, params)

    if req.status_code == 200:
        print('tweet succeed!')
    else:
        print('tweet failed')
        
def tweet_search(search_word, num, oauth):
    url = 'https://api.twitter.com/1.1/search/tweets.json'
    params = {
        'q': search_word,
        'count' : num,
        'result_type' : 'recent',
        'exclude': 'retweets',
        'lang' : 'ja'
        }
    responce = oauth.get(url, params=params)
    if responce.status_code != 200:
        print("Error code: %d" %(responce.status_code))
        return None
    tweets = json.loads(responce.text)
    return tweets

keysfile = '../../twitter_API/key/keys.json'
keys = json.load(open(keysfile))
oauth = create_oauth_session(keys)

In [3]:
def remove_emoji(text):
    return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI['en'])

def remove_url(text):
    return re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)

In [4]:
search = tweet_search("ビットコイン", 100, oauth)
df_search = DataFrame.from_dict(search['statuses'])
tweets = df_search['text'].tolist()

In [5]:
tweets[:10]

['年内に3倍以上？👀🔥ビットコインもイーサリアムも急騰中！XRP最新情報＆熱いオルトコインChainlinkとFrontierが提携！Dogeコインの謎の急成長とNFT最新情報！（動画）… https://t.co/JFLLfQ37Ax',
 'イーサリアムがトレンド入りしてる。\n年末のビットコインの伸びを鑑みると、3,400ドルがいったんの節目だと思ってる。\n\n#100万行くまで売りません\n#ETH #イーサリアム',
 'ビットコイン、買いたい気持ち、焦る気持ちが値段が上がるにつれて出てくるけど、我慢が大事ですよね',
 'BAT ベーシックアテンショントークンは\nPoL @PoL_techtec のサイトで学べます！\n\n仮想通貨は値動きの上下だけではなく、\n使用されている技術について見識を深めるのもよきですね！\n\nビットコイン や イーサリアムもP… https://t.co/dsY7F5o1RG',
 '@UNOu3JaOmk4Tt9i ビットコイン😁？',
 'ビットコインの最後の半減期後ってどうなるんですか？',
 'ビットコインまためちゃくちゃ巻き返してるね。1万円分だけしかないけど、せっかくだから何があっても置いておこうかな…。',
 '@fuyuto_topics ウォレットを始めた頃ビットコインがもう高めだったので低いイーサムに絞りました。\nこれからもポイントは下がればビットコインとイーサムを買い続けます。',
 'ビットコイン買ってる人！\nどこのサイトで買ってる？？',
 '短期トレードのイメージ\n\n東大に行ける人がいるように、可能性は否定できないし\nTwitterにはそれ以上の超人がいっぱいる。ビットコインみたいな可能性もあるわけだしな。人生良きも悪くも何があるか分からん！ https://t.co/11YFRcdnAF']

In [6]:
texts = []
for tweet in tweets:
    text = tweet
    text = remove_emoji(text)
    text = remove_url(text)
    texts.append(text)

In [7]:
texts[:10]

['年内に3倍以上？ビットコインもイーサリアムも急騰中！XRP最新情報＆熱いオルトコインChainlinkとFrontierが提携！Dogeコインの謎の急成長とNFT最新情報！（動画）… ',
 'イーサリアムがトレンド入りしてる。\n年末のビットコインの伸びを鑑みると、3,400ドルがいったんの節目だと思ってる。\n\n#100万行くまで売りません\n#ETH #イーサリアム',
 'ビットコイン、買いたい気持ち、焦る気持ちが値段が上がるにつれて出てくるけど、我慢が大事ですよね',
 'BAT ベーシックアテンショントークンは\nPoL @PoL_techtec のサイトで学べます！\n\n仮想通貨は値動きの上下だけではなく、\n使用されている技術について見識を深めるのもよきですね！\n\nビットコイン や イーサリアムもP… ',
 '@UNOu3JaOmk4Tt9i ビットコイン？',
 'ビットコインの最後の半減期後ってどうなるんですか？',
 'ビットコインまためちゃくちゃ巻き返してるね。1万円分だけしかないけど、せっかくだから何があっても置いておこうかな…。',
 '@fuyuto_topics ウォレットを始めた頃ビットコインがもう高めだったので低いイーサムに絞りました。\nこれからもポイントは下がればビットコインとイーサムを買い続けます。',
 'ビットコイン買ってる人！\nどこのサイトで買ってる？？',
 '短期トレードのイメージ\n\n東大に行ける人がいるように、可能性は否定できないし\nTwitterにはそれ以上の超人がいっぱいる。ビットコインみたいな可能性もあるわけだしな。人生良きも悪くも何があるか分からん！ ']

In [8]:
from janome.tokenizer import Tokenizer

In [9]:
t = Tokenizer()
worddic = {}

In [10]:
for line in texts:
    malist = t.tokenize(line)
    for w in malist:
        word = w.surface
        part = w.part_of_speech
        if part.find('名詞') < 0:
            continue
        if not word in worddic:
            worddic[word] = 0
        worddic[word] += 1

In [21]:
trend_words = ''
keys = sorted(worddic.items(), key=lambda x:x[1], reverse=True)
for word, cnt in keys[:50]:
    trend_words += word + ' '
    print('{0}({1})'.format(word, cnt))

コイン(133)
ビット(127)
#(45)
円(44)
通貨(39)
仮想(32)
万(22)
イーサリアム(19)
,(18)
の(17)
@(14)
3(13)
人(13)
)(13)
BTC(13)
こと(12)
1(10)
ん(9)
((9)
投資(9)
登録(9)
/(9)
.(9)
価格(9)
分(8)
取引(8)
的(8)
方(8)
中(7)
暗号(7)
時(7)
:(7)
現在(7)
+(7)
%(7)
売買(7)
以上(6)
ETH(6)
_(6)
サイト(6)
みたい(6)
資産(6)
まとめ(6)
21(6)
私(6)
年(6)
入金(6)
自動(6)
000(6)
後(5)


In [14]:
def word_cloud(words, image_file, font_path='C:\Windows/Fonts/MSGOTHIC.TTC'):
    wordcloud = WordCloud(
            background_color='white', font_path=font_path,   ## 日本語フォントを指定
            width=800, height=400).generate(words)

    ## 結果をファイルへ保存
    wordcloud.to_file(image_file)

    return True

In [22]:
word_cloud(trend_words, 'word_cloud_test.png')

True

In [26]:
search_word = 'ビットコイン'
search_num = 10
trend_word_num = 50
tweet_text = 'search word: {}\nsearch tweets num: {}\ntrend words num: {}'.format(search_word, search_num, trend_word_num)
tweet_text

'search word: ビットコイン\nsearch tweets num: 10\ntrend words num: 50'

In [27]:
print(tweet_text)

search word: ビットコイン
search tweets num: 10
trend words num: 50


In [30]:
tweet_image(oauth, tweet_text, 'word_cloud_test.png')

Media ID: 1389200235184877569
tweet succeed!
